# Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

In this Colab, we present a step-by-step guide on how to fine-tune Whisper 
for any multilingual ASR dataset using Hugging Face 🤗 Transformers. This is a 
more "hands-on" version of the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper). 
For a more in-depth explanation of Whisper, the Common Voice dataset and the theory behind fine-tuning, the reader is advised to refer to the blog post.

## Introduction

Whisper is a pre-trained model for automatic speech recognition (ASR) 
published in [September 2022](https://openai.com/blog/whisper/) by the authors 
Alec Radford et al. from OpenAI. Unlike many of its predecessors, such as 
[Wav2Vec 2.0](https://arxiv.org/abs/2006.11477), which are pre-trained 
on un-labelled audio data, Whisper is pre-trained on a vast quantity of 
**labelled** audio-transcription data, 680,000 hours to be precise. 
This is an order of magnitude more data than the un-labelled audio data used 
to train Wav2Vec 2.0 (60,000 hours). What is more, 117,000 hours of this 
pre-training data is multilingual ASR data. This results in checkpoints 
that can be applied to over 96 languages, many of which are considered 
_low-resource_.

When scaled to 680,000 hours of labelled pre-training data, Whisper models 
demonstrate a strong ability to generalise to many datasets and domains.
The pre-trained checkpoints achieve competitive results to state-of-the-art 
ASR systems, with near 3% word error rate (WER) on the test-clean subset of 
LibriSpeech ASR and a new state-of-the-art on TED-LIUM with 4.7% WER (_c.f._ 
Table 8 of the [Whisper paper](https://cdn.openai.com/papers/whisper.pdf)).
The extensive multilingual ASR knowledge acquired by Whisper during pre-training 
can be leveraged for other low-resource languages; through fine-tuning, the 
pre-trained checkpoints can be adapted for specific datasets and languages 
to further improve upon these results. We'll show just how Whisper can be fine-tuned 
for low-resource languages in this Colab.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 1:</b> Whisper model. The architecture 
follows the standard Transformer-based encoder-decoder model. A 
log-Mel spectrogram is input to the encoder. The last encoder 
hidden states are input to the decoder via cross-attention mechanisms. The 
decoder autoregressively predicts text tokens, jointly conditional on the 
encoder hidden states and previously predicted tokens. Figure source: 
<a href="https://openai.com/blog/whisper/">OpenAI Whisper Blog</a>.</figcaption>
</figure>

The Whisper checkpoints come in five configurations of varying model sizes.
The smallest four are trained on either English-only or multilingual data.
The largest checkpoint is multilingual only. All nine of the pre-trained checkpoints 
are available on the [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper). The 
checkpoints are summarised in the following table with links to the models on the Hub:

| Size   | Layers | Width | Heads | Parameters | English-only                                         | Multilingual                                      |
|--------|--------|-------|-------|------------|------------------------------------------------------|---------------------------------------------------|
| tiny   | 4      | 384   | 6     | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)  |
| base   | 6      | 512   | 8     | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)   |
| small  | 12     | 768   | 12    | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)  |
| medium | 24     | 1024  | 16    | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium) |
| large  | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)  |

For demonstration purposes, we'll fine-tune the multilingual version of the 
[`"small"`](https://huggingface.co/openai/whisper-small) checkpoint with 244M params (~= 1GB). 
As for our data, we'll train and evaluate our system on a low-resource language 
taken from the [Common Voice](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0)
dataset. We'll show that with as little as 8 hours of fine-tuning data, we can achieve 
strong performance in this language.

------------------------------------------------------------------------

\\({}^1\\) The name Whisper follows from the acronym “WSPSR”, which stands for “Web-scale Supervised Pre-training for Speech Recognition”.

## Prepare Environment

We can verify that we've been assigned a GPU and view its specifications:

In [1]:
# Run the following command separately before running the py version of this notebook!!
#git config --global credential.helper store
#huggingface-cli login

In [2]:
import os

abs_path = os.path.abspath('.')
base_dir = os.path.dirname(os.path.dirname(abs_path))

os.environ['TRANSFORMERS_CACHE'] = os.path.join(base_dir, 'models_cache')
os.environ['TRANSFORMERS_OFFLINE'] = '0'
os.environ['HF_DATASETS_CACHE'] = os.path.join(base_dir, 'datasets_cache')
os.environ['HF_DATASETS_OFFLINE'] = '0'

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('NOT connected to a GPU!. Will use CPU!')
else:
  print('Connected to a GPU!')

Connected to a GPU!


In [4]:
# sudo add-apt-repository -y ppa:jonathonf/ffmpeg-4
# sudo apt update
# sudo apt install -y ffmpeg
# !pip install huggingface_hub
# !pip install ipywidgets
# !pip install git+https://github.com/huggingface/datasets
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install evaluate>=0.3.0
# !pip install jiwer
# !pip install bnunicodenormalizer

Linking the notebook to the Hugging Face Hub is straightforward - it simply requires entering your 
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [5]:
from huggingface_hub import notebook_login

notebook_login()

## Variables

In [6]:
# model_name = "openai/whisper-large-v2"
model_name = "openai/whisper-tiny"
language = "bn"
task = "transcribe" # transcribe or translate

### Training Args

In [7]:
output_dir = "./" 
overwrite_output_dir = True
# max_steps = 60000 
max_steps = 10 
# per_device_train_batch_size = 4 
per_device_train_batch_size = 1 
# per_device_eval_batch_size = 2 
per_device_eval_batch_size = 1 
# gradient_accumulation_steps = 8 
gradient_accumulation_steps = 1 
gradient_checkpointing = False 
evaluation_strategy ="steps" 
eval_steps =2 
# eval_steps = 1000 
save_strategy = "steps" 
# save_steps = 1000 
save_steps = 2 
save_total_limit = 5 
learning_rate = 1e-5 
# warmup_steps = 5000 
warmup_steps = 1 
# logging_steps = 25
logging_steps = 1
weight_decay = 0.01 
load_best_model_at_end = True 
metric_for_best_model = "wer" 
greater_is_better = False 
# bf16 = True 
bf16 = False
# tf32 = True 
tf32 = False
generation_max_length = 225 
report_to = ["tensorboard"] 
predict_with_generate = True
# push_to_hub = True
push_to_hub = False
freeze_feature_encoder = False 

## Load & Merge Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple. 
We can download and prepare the Common Voice splits in just one line of code. 

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

Since Bangla is very low-resource, we'll combine the `train` and `validation` 
splits to give approximately 8 hours of training data. We'll use the 4 hours 
of `test` data as our held-out test set:

In [8]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
google_fleurs = DatasetDict()
openslr = DatasetDict()
## commonvoice_11.0 + google_fleurs + openslr53
my_dataset = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "bn", split="train+validation", cache_dir=os.path.join(base_dir, 'datasets_cache'))
google_fleurs["train"] = load_dataset("google/fleurs", "bn_in", split="train+validation", cache_dir=os.path.join(base_dir, 'datasets_cache'))
openslr = load_dataset("openslr", "SLR53", cache_dir=os.path.join(base_dir, 'datasets_cache'))

common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "bn", split="test", cache_dir=os.path.join(base_dir, 'datasets_cache'))
google_fleurs["test"] = load_dataset("google/fleurs", "bn_in", split="test", cache_dir=os.path.join(base_dir, 'datasets_cache'))

Found cached dataset common_voice_11_0 (e:/datasets_cache/mozilla-foundation___common_voice_11_0/bn/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f)
Found cached dataset fleurs (e:/datasets_cache/google___fleurs/bn_in/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)
Found cached dataset openslr (e:/datasets_cache/openslr/SLR53/0.0.0/cd9de085f77465c1cb7d394e748a3a5b80fad2049e7af92f67baf04b894f9567)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset common_voice_11_0 (e:/datasets_cache/mozilla-foundation___common_voice_11_0/bn/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f)
Found cached dataset fleurs (e:/datasets_cache/google___fleurs/bn_in/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


Here's how we will merge the three datasets:

- Most ASR datasets only provide input audio samples (`audio`) and the 
corresponding transcribed text (`sentence`). Any additional 
metadata information, such as `accent` and `locale`, are disregarded for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information using the `remove_columns` method

- We _downsample_ any audio data down to 16kHz prior to passing it to the Whisper feature extractor, as 16kHz is expected by the Whisper model. We'll set the audio inputs to the correct sampling rate using dataset's 
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place, 
but rather signals to `datasets` to resample audio samples _on the fly_ the 
first time that they are loaded

In [9]:
#small subset for testing

common_voice['train']  = common_voice['train'].select(range(50))
common_voice['test']  = common_voice['test'].select(range(50))
google_fleurs['train']  = google_fleurs['train'].select(range(50))
google_fleurs['test']  = google_fleurs['test'].select(range(50))
openslr['train'] = openslr['train'].select(range(50))

In [10]:
from datasets import concatenate_datasets, Audio

sampling_rate = 16000

## resample to specified sampling rate
common_voice = common_voice.cast_column("audio", Audio(sampling_rate))
google_fleurs = google_fleurs.cast_column("audio", Audio(sampling_rate))
openslr = openslr.cast_column("audio", Audio(sampling_rate))

## normalise columns to ["audio", "sentence"]
common_voice = common_voice.remove_columns(
    set(common_voice['test'].features.keys()) - {"audio", "sentence"}
)

google_fleurs = google_fleurs.rename_column("raw_transcription", "sentence")
google_fleurs = google_fleurs.remove_columns(
    set(google_fleurs['test'].features.keys()) - {"audio", "sentence"}
)

openslr = openslr.remove_columns(
    set(openslr['train'].features.keys()) - {"audio", "sentence"}
)

## merge the three datasets
my_dataset['train'] = concatenate_datasets([common_voice['train'], google_fleurs['train'], openslr['train']]) #for linux
# my_dataset['train'] = concatenate_datasets([google_fleurs['train'], openslr['train']]) #for windows no commonvoice as it requires ffmpeg-4
my_dataset['test'] = concatenate_datasets([common_voice['test'], google_fleurs['test']]) #for linux
# my_dataset['test'] = concatenate_datasets([google_fleurs['test']]) #for windows no commonvoice as it requires ffmpeg-4

#shuffle train set with seed=42
my_dataset['train'] = my_dataset['train'].shuffle(seed=42)

In [16]:
# i=49
# common_voice['train'][i]['sentence'], google_fleurs['train'][i]['sentence'], openslr['train'][i]['sentence']

('পিটম্যান ইঙ্গিত দিয়েছিলেন যে আগামী সপ্তাহের আগে অবস্থার উন্নতি হবে না।',
 'প্রায়ই ‘এককাট্টা শ্রেণী’ হিসেবে নিয়েছেন')

### Examining features and dtypes are aligned or not

In [11]:
# google_fleurs['train'].features
# common_voice['train'].features
# openslr['train'].features
my_dataset['train'].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [13]:
# openslr['train'][0]['audio']['array'].dtype
# google_fleurs['train'][0]['audio']['array'].dtype
# common_voice['test'][1]['audio']['array'].dtype

## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages: 
1) A feature extractor which pre-processes the raw audio-inputs
2) The model which performs the sequence-to-sequence mapping 
3) A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer, 
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer) 
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [14]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)

### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For Hindi, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to 
specify the target language and the task. These arguments inform the 
tokenizer to prefix the language and task tokens to the start of encoded 
label sequences:

In [15]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name, language=language, task=task)

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_ 
both into a single `WhisperProcessor` class. This processor object 
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`, 
and can be used on the audio inputs and model predictions as required. 
In doing so, we only need to keep track of two objects during training: 
the `processor` and the `model`:

In [16]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name, language=language, task=task)

### Prepare Data

Let's print the first example of the Common Voice dataset to see 
what form the data is in:

In [17]:
print(my_dataset["train"][0])

{'audio': {'path': 'E:\\datasets_cache\\downloads\\extracted\\f996294c90897704ab75685bd55cdb07f545c26d5a7520235ad785bd27c4444e\\asr_bengali\\data\\00\\0001fd54d9.flac', 'array': array([0.0012207 , 0.00262451, 0.00354004, ..., 0.00616455, 0.00616455,
       0.003479  ], dtype=float32), 'sampling_rate': 16000}, 'sentence': 'আপনার গল্প ছাড়া'}


We'll define our pre-processing strategy. We advise that you **do not** lower-case the transcriptions or remove punctuation unless mixing different datasets. This will enable you to fine-tune Whisper models that can predict punctuation and casing. Later, you will see how we can evaluate the predictions without punctuation or casing, so that the models benefit from the WER improvement obtained by normalising the transcriptions while still predicting fully formatted transcriptions.

In [18]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from bnunicodenormalizer import Normalizer

do_lower_case = False
do_remove_punctuation = False
do_bangla_unicode_normalization = True

normalizer = BasicTextNormalizer()
bangla_normalizer = Normalizer(allow_english=True)

Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We perform any optional pre-processing (lower-case or remove punctuation or normalize Bangla to Unicode characters).
4. We encode the transcriptions to label ids through the use of the tokenizer.

In [19]:
def prepare_dataset(batch):
    # load and (possibly) resample audio data to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    
    # optional pre-processing steps
    transcription = batch["sentence"]
    if do_lower_case:
        transcription = transcription.lower()
    if do_remove_punctuation:
        transcription = normalizer(transcription).strip()
    if do_bangla_unicode_normalization:
        _words = [bangla_normalizer(word)['normalized'] for word in transcription.split()]
        transcription = " ".join([word for word in _words if word is not None])
    
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(transcription).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [20]:
# common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)
my_dataset = my_dataset.map(prepare_dataset, num_proc=1)

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/50 [00:00<?, ?ex/s]

Finally, we filter any training data with audio samples longer than 30s. These samples would otherwise be truncated by the Whisper feature-extractor which could affect the stability of training. We define a function that returns `True` for samples that are less than 30s, and `False` for those that are longer:

In [21]:
max_input_length = 30.0
min_input_length = 1.0

def is_audio_in_length_range(length):
    return length < max_input_length and length > min_input_length

We apply our filter function to all samples of our training dataset through 🤗 Datasets' `.filter` method:

In [22]:
my_dataset["train"] = my_dataset["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

  0%|          | 0/1 [00:00<?, ?ba/s]

### Save & Cleanup Cache Files

In [23]:
my_dataset.save_to_disk(os.path.join(base_dir, "datasets_cache"))

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/99 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
## Returns the number of removed cache files
my_dataset.cleanup_cache_files()
common_voice.cleanup_cache_files()
google_fleurs.cleanup_cache_files()
openslr.cleanup_cache_files()

In [26]:
from datasets import load_from_disk
my_dataset = load_from_disk(os.path.join(base_dir, "datasets_cache"))

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline. 
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it 
to transcribe speech in Hindi.

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it 
treats the `input_features` and `labels` independently: the  `input_features` must be 
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram 
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens 
are then replaced by `-100` so that these tokens are **not** taken into account when 
computing the loss. We then cut the BOS token from the start of the label sequence as we 
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the 
feature extractor and the tokenizer operations:

In [28]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [29]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing 
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [30]:
import evaluate

metric = evaluate.load("wer", cache_dir=os.path.join(base_dir, "metrics_cache"))

We then simply have to define a function that takes our model 
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the 
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels. 
Here, we have the option of evaluating with the 'normalised' transcriptions 
and predictions. We recommend you set this to `True` to benefit from the WER 
improvement obtained by normalising the transcriptions.

In [31]:
# evaluate with the 'normalised' WER
do_normalize_eval = True

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this 
is trivial through use of 🤗 Transformers!

In [32]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

e:\Govt_Speech_Demo\env\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\models_cache. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/151M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)). Set `use_cache` to False since we're using gradient checkpointing, and the two are incompatible:

In [35]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
if gradient_checkpointing:
    model.config.use_cache = False
if freeze_feature_encoder:
    model.freeze_feature_encoder()

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [39]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    max_steps=max_steps,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    evaluation_strategy=evaluation_strategy,
    eval_steps=eval_steps,
    save_strategy=save_strategy,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    logging_steps=logging_steps,
    weight_decay=weight_decay,
    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    greater_is_better=greater_is_better,
    bf16=bf16,
    tf32=tf32,
    generation_max_length=generation_max_length,
    report_to=report_to,
    predict_with_generate=predict_with_generate,
    push_to_hub=push_to_hub,
)

**Note**: if one does not want to upload the model checkpoints to the Hub, 
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [42]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=my_dataset["train"],
    eval_dataset=my_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [43]:
# processor.save_pretrained(training_args.output_dir)

Feature extractor saved in ./preprocessor_config.json
tokenizer config file saved in ./tokenizer_config.json
Special tokens file saved in ./special_tokens_map.json
added tokens file saved in ./added_tokens.json


### Training

Training will take approximately 5-10 hours depending on your GPU. The peak GPU memory for the given training configuration is approximately 36GB. 
Depending on your GPU, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training. 
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2 
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

In [44]:
train_result = trainer.train()
## resume from the latest checkpoint
# train_result = trainer.train(resume_from_checkpoint=True)
## resume training from the specific checkpoint in the directory passed
# train_result = trainer.train(resume_from_checkpoint="checkpoint-4000")

The following columns in the training set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: input_length, sentence, audio. If input_length, sentence, audio are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
e:\Govt_Speech_Demo\env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 99
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 10
  Number of trainable parameters = 37760640


  0%|          | 0/10 [00:00<?, ?it/s]

{'loss': 4.1199, 'learning_rate': 1e-05, 'epoch': 0.01}


The following columns in the evaluation set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: input_length, sentence, audio. If input_length, sentence, audio are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


{'loss': 5.1783, 'learning_rate': 8.888888888888888e-06, 'epoch': 0.02}


...

  0%|          | 0/50 [00:00<?, ?it/s]

Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Configuration saved in ./checkpoint-10\generation_config.json


{'eval_loss': 2.2146687507629395, 'eval_wer': 117.97314854915548, 'eval_runtime': 165.3566, 'eval_samples_per_second': 0.302, 'eval_steps_per_second': 0.302, 'epoch': 0.1}


Model weights saved in ./checkpoint-10\pytorch_model.bin
Feature extractor saved in ./checkpoint-10\preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./checkpoint-2 (score: 117.58336942399308).


{'train_runtime': 836.8903, 'train_samples_per_second': 0.012, 'train_steps_per_second': 0.012, 'train_loss': 3.100829911231995, 'epoch': 0.1}


### Saving Metrics & Model

In [45]:
if not os.path.exists("best_model"):
    os.makedirs("best_model")

processor.save_pretrained("best_model")
trainer.save_model("best_model")


Feature extractor saved in ./best_model\preprocessor_config.json
tokenizer config file saved in ./best_model\tokenizer_config.json
Special tokens file saved in ./best_model\special_tokens_map.json
added tokens file saved in ./best_model\added_tokens.json
Saving model checkpoint to ./best_model
Configuration saved in ./best_model\config.json
Configuration saved in ./best_model\generation_config.json
Model weights saved in ./best_model\pytorch_model.bin
Feature extractor saved in ./best_model\preprocessor_config.json


In [46]:
metrics = train_result.metrics
trainer.save_metrics("train", metrics)
trainer.save_state()

In [47]:
metrics = trainer.evaluate(
    metric_key_prefix="eval",
    max_length=training_args.generation_max_length,
    num_beams=training_args.generation_num_beams,
)
trainer.save_metrics("eval", metrics)

The following columns in the evaluation set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: input_length, sentence, audio. If input_length, sentence, audio are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


  0%|          | 0/50 [00:00<?, ?it/s]

### Push to Hub

We can label our checkpoint with the `whisper-event` tag on push by setting the appropriate key-word arguments (kwargs):

In [ ]:
kwargs = {
    "dataset_tags": ["mozilla-foundation/common_voice_11_0", "google/fleurs", "openslr"],
    "dataset": "common-voice-11+google-fleurs+openslr53",  # a 'pretty' name for the training dataset
    "language": "bn",
    "model_name": "Whisper Small - Mohammed Rakib",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "whisper-event",
}

The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

In [ ]:
trainer.push_to_hub(**kwargs)

## Closing Remarks

In this blog, we covered a step-by-step guide on fine-tuning Whisper for multilingual ASR 
using 🤗 Datasets, Transformers and the Hugging Face Hub. For more details on the Whisper model, the Common Voice dataset and the theory behind fine-tuning, refere to the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper). If you're interested in fine-tuning other 
Transformers models, both for English and multilingual ASR, be sure to check out the 
examples scripts at [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).